# Stock Predict Model

## Import Modules

In [361]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# For Graph
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Yahoo API
import yfinance as yf

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

## Configurations

In [362]:
device = 'cpu' if torch.backends.mps.is_available() else 'cpu'
scaler = MinMaxScaler()
config = {
    'seed': 666999,
    'select_all': False,
    'valid_ratio': 0.2,
    'test_ratio': 0.2,
    'n_epochs': 3000,
    'batch_size': 256,
    'learning_rate': 1e-3,
    'early_stop': 400,
    'save_path': './models/stock.ckpt'
}

## Utility Functions

In [392]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

def predict(test_loader, model, device):
    model.eval()
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():

            pred = model(x)
            pred = scaler.inverse_transform(pred.detach().cpu().numpy().reshape(-1, 1))
            preds.append(pred)
    preds = [torch.from_numpy(pred) for pred in preds]
    preds = torch.cat(preds, dim=0).numpy()
    return preds

def single_predict(single, model, device):
    print(f"single: {single}")
    model.eval()
    single = np.array(single)
    single = torch.from_numpy(single).unsqueeze(0).to(device)
    single = single.to(torch.float32)

    with torch.no_grad():
        pred = model(single)
        # pred = scaler.inverse_transform(pred.detach().cpu().numpy().reshape(-1, 1))
        print(f"pred: {pred}")
        return pred

## Dataset

In [364]:
class Stock_Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        self.y = y if y is None else torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

## Neural Network Model

In [374]:
class Stock_Model(nn.Module):
    def __init__(self, input_dim):
        super(Stock_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

## Feature Selection

In [366]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:, :-1], valid_data[:, :-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2] # TODO: Select suitable feature columns.

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

## Training Loop

In [367]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7)

    writer = SummaryWriter()

    if not os.path.isdir('./models'):
        os.mkdir('./models')

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train()
        loss_record = []

        train_pbar = tqdm(train_loader, position=0, leave=True)
        for x, y in train_pbar:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            step += 1
            loss_record.append(loss.detach().item())

            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})
        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval()
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## Dataloader

In [368]:
same_seed(config['seed'])
# re-index
index = ["Open", "High", "Low", "Close", "Volume", "Adj Close"]
# train_data size: 5255 x 7 (id + 6 features From 2000-01-01 to 2020-12-31)
# test_data size: 577 x 7 (id + 6 features From 2021-01-01 to 2023-12-31)
# train_data, test_data = pd.DataFrame(yf.download('2330.TW', start='2000-01-01', end='2020-12-31'))[index].values, \
#                         pd.DataFrame(yf.download('2330.TW', start='2021-01-01', end='2023-12-31'))[index[:-1]].values
raw_data = pd.DataFrame(yf.download('2330.TW', start='2000-01-01', end='2023-12-31'))
train_data, test_data = train_test_split(
    raw_data[index].values,
    test_size=config['test_ratio'],
    random_state=config['seed']
)

train_data, valid_data = train_test_split(
    train_data,
    test_size=config['valid_ratio'],
    random_state=config['seed']
)

# train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

print(f"""
Before Normalization
x_train size: {x_train.shape}
x_valid size: {x_valid.shape}
x_test size: {x_test.shape}
y_train size: {y_train.shape}
y_valid size: {y_valid.shape}""")

# Normalization
x_train = scaler.fit_transform(x_train)
x_valid = scaler.fit_transform(x_valid)
x_test = scaler.fit_transform(x_test)
y_train = scaler.fit_transform(y_train.reshape(-1, 1))
y_valid = scaler.fit_transform(y_valid.reshape(-1, 1))

print(f"""
After Normalization
x_train size: {x_train.shape}
x_valid size: {x_valid.shape}
x_test size: {x_test.shape}
y_train size: {y_train.shape}
y_valid size: {y_valid.shape}""")

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = Stock_Dataset(x_train, y_train), \
                                             Stock_Dataset(x_valid, y_valid), \
                                             Stock_Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

[*********************100%***********************]  1 of 1 completed
train_data size: (3734, 6)
valid_data size: (934, 6)
test_data size: (1168, 6)

Before Normalization
x_train size: (3734, 3)
x_valid size: (934, 3)
x_test size: (1168, 3)
y_train size: (3734,)
y_valid size: (934,)

After Normalization
x_train size: (3734, 3)
x_valid size: (934, 3)
x_test size: (1168, 3)
y_train size: (3734, 1)
y_valid size: (934, 1)
number of features: 3


## Training

In [375]:
model = Stock_Model(input_dim=x_train.shape[1]).to(device)
trainer(train_loader, valid_loader, model, config, device)

# with torch.profiler.profile(
#     activities=[torch.profiler.ProfilerActivity.CPU],
#     schedule = torch.profiler.schedule(
#         wait=0,
#         warmup=0,
#         active=1
#     ),
#     on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
# ) as prof:
#     trainer(train_loader, valid_loader, model, config, device)
#     prof.step()

  0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch [1/3000]:   0%|          | 0/15 [00:00<?, ?it/s, loss=0.199]/usr/local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([150, 1])) that is different to the input size (torch.Size([150])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch [1/3000]: 100%|██████████| 15/15 [00:00<00:00, 148.09it/s, loss=0.169]
/usr/local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([166, 1]))

Epoch [1/3000]: Train loss: 0.2076, Valid loss: 0.1968
Saving model with loss 0.197...


Epoch [2/3000]: 100%|██████████| 15/15 [00:00<00:00, 164.56it/s, loss=0.124]


Epoch [2/3000]: Train loss: 0.1545, Valid loss: 0.1519
Saving model with loss 0.152...


Epoch [3/3000]: 100%|██████████| 15/15 [00:00<00:00, 138.17it/s, loss=0.0903]


...

Epoch [709/3000]: Train loss: 0.0604, Valid loss: 0.0688


Epoch [710/3000]: 100%|██████████| 15/15 [00:00<00:00, 212.46it/s, loss=0.0716]


Epoch [710/3000]: Train loss: 0.0603, Valid loss: 0.0685


Epoch [711/3000]: 100%|██████████| 15/15 [00:00<00:00, 210.13it/s, loss=0.0741]


Epoch [711/3000]: Train loss: 0.0604, Valid loss: 0.0658

Model is not improving, so we halt the training session.


In [370]:
# print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))


In [371]:
%tensorboard --logdir=./log/

## Plot learning curves with tensorboard

In [372]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

## Testing

In [373]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = Stock_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')

100%|██████████| 5/5 [00:00<00:00, 474.84it/s]


In [394]:
model = Stock_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))

single_predict([650,667,633], model, device)

single: [650, 667, 633]
pred: tensor([27.5125])


tensor([27.5125])